# Deep Homography Net

In [1]:
import glob
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC'
tf.Session(config = config)

C:\Users\TWright\Anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7363920774113876615
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2893365248
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9601633562248925720
physical_device_desc: "device: 0, name: GeForce GTX 970, pci bus id: 0000:01:00.0, compute capability: 5.2"
]


## Data Wrangling

Read the directory

In [ ]:
# todo

## HomographyNet

Regression variant

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Input, Activation
from keras.layers.convolutional import Convolution2D
from keras.optimizers import Adam
import keras.backend as K

def euclidean_distance(y_true, y_pred):
    return K.sqrt(K.maximum(K.sum(K.square(y_pred - y_true), axis=-1, keepdims=True), K.epsilon()))

def conv_block(m, filters):
    m = Convolution2D(filters, 3, 3, subsample=(1, 1), border_mode='same', activation='relu')(m)
    m = Convolution2D(filters, 3, 3, subsample=(1, 1), border_mode='same', activation='relu')(m)
    return MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool2')(m)

def homography_regression_model(input_dims):
    input_shape=(*input_dims, 2)
    
    input_layer = Input(shape=input_shape, name='input layer')
    
    x = conv_block(input_layer, 64)
    x = conv_block(x,           64)
    x = conv_block(x,           128)
    x = conv_block(x,           128)
    
    x = Convolution2D(128, 3, 3, subsample=(1, 1), border_mode='same', activation='relu')(x)
    x = Convolution2D(128, 3, 3, subsample=(1, 1), border_mode='same', activation='relu')(x)
    
    x = Flatten()(x)
    x = Dense(1024, name='FC 1024')(x)
    out = Dense(8, name='output')(x)
    
    return Model(input=input_img, output=[out])

Using TensorFlow backend.


In [3]:
my_model = homography_regression_model()
my_model = model.compile(optimizer=Adam(lr=1e-3), loss=euclidean_distance)
my_model.summary()

TypeError: homography_regression_model() missing 1 required positional argument: 'input_dims'

## Data Generators

We generate the "seemingly infinite training data" on the fly by using Keras' DataGenerators.

In [4]:
# todo

## Training